# Overview

I do not have too much experience of hands on tensorflow. So, let's see how to use it pre-process data, create model and do the training. Before the coding part, let's see the common framework and mechanism in modern LLMs.

## Encoder-Decoder architecture

Sequence-to-sequence models have revolutionized the way we approach language tasks in NLP. The core idea is to map a sequence of inputs (like words in a sentence) to a sequence of outputs (like translated words in another language). This mapping is achieved through two main components: the encoder and the decoder, often implemented using Long Short-Term Memory (LSTM) networks or Gated Recurrent Units (GRUs).

## Attention Mechanism

While the encoder-decoder architecture provides a robust framework for sequence mapping, it’s not without limitations. One key issue is the reliance on a fixed-length context vector to encode the entire input sequence, which can be problematic for long sequences.

Attention Mechanism computes a weight distribution (or attention scores) that determines the importance of each input element for each output.

* Attention Scores
* Context Vector
* Decoder’s Input

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def data_preprocessor(source_sentences, target_sentences):
    
    source_tokenizer = Tokenizer()
    source_tokenizer.fit_on_texts(source_sentences)
    source_sequences = source_tokenizer.texts_to_sequences(source_sentences)
    source_padded = pad_sequences(source_sequences, padding='post')
      
    target_tokenizer = Tokenizer()
    target_tokenizer.fit_on_texts(target_sentences)
    target_sequences = target_tokenizer.texts_to_sequences(target_sentences)
    target_padded = pad_sequences(target_sequences, padding='post')
    
    return source_padded, target_padded, source_tokenizer, target_tokenizer

english_sentences = ['hello', 'world', 'how are you', 'I am fine', 'have a good day']
spanish_sentences = ['hola', 'mundo', 'cómo estás', 'estoy bien', 'ten un buen día']
input_texts, target_texts, source_tokenizer, target_tokenizer = data_preprocessor(english_sentences, spanish_sentences)

In [7]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate
from tensorflow.keras.layers import AdditiveAttention as Attention
from tensorflow.keras.models import Model

# Model parameters
embedding_dim = 256
latent_dim = 512
num_encoder_tokens = len(source_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Attention Layer
attention = Attention()
attention_output = attention([decoder_outputs, encoder_outputs])

# Concatenating attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_output])

# Dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
from tensorflow.keras.utils import to_categorical

decoder_target_data = to_categorical(target_texts, num_decoder_tokens)
model.fit([input_texts, target_texts], decoder_target_data, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0000e+00 - loss: 2.3969 - val_accuracy: 0.0000e+00 - val_loss: 2.4097
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.6250 - loss: 2.2366 - val_accuracy: 0.0000e+00 - val_loss: 2.4232
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.6250 - loss: 2.0541 - val_accuracy: 0.0000e+00 - val_loss: 2.4432
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.6250 - loss: 1.7782 - val_accuracy: 0.0000e+00 - val_loss: 2.4780
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.6250 - loss: 1.4115 - val_accuracy: 0.0000e+00 - val_loss: 2.5372
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.6250 - loss: 1.1964 - val_accuracy: 0.0000e+00 - val_loss: 2.5743
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.6250 - loss: 1.1376 - val_accuracy: 0.0000e+00 - val_loss: 2.6060
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy: 0.6250 - loss: 1.0893 - val

In [9]:
# Encoder Inference Model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder Inference Model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

ValueError: Input 0 of layer "dense_1" is incompatible with the layer: expected axis -1 of input shape to have value 1024, but received input with shape (None, None, 512)

In [ ]:
def translate(input_text):
    # Tokenize and pad the input sequence
    input_seq = source_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=input_texts.shape[1], padding='post')

    # Get the encoder states
    states_value = encoder_model.predict(input_seq)

    # Generate an empty target sequence of length 1
    target_seq = np.zeros((1, 1))

    # Populate the first character of the target sequence with the start character
    target_seq[0, 0] = target_tokenizer.word_index['start']  # Assuming 'start' is the start token

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_tokenizer.index_word[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length or find stop token.
        if (sampled_char == 'end' or len(decoded_sentence) > 50):  # Assuming 'end' is the end token
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Example usage
translated_sentence = translate("hello")
print(translated_sentence)

# Acknowledge
* https://medium.com/@mervebdurna/exploring-seq2seq-encoder-decoder-and-attention-mechanisms-in-nlp-theory-and-practice-9b1022cf50b4
* https://github.com/mervebdurna/10-days-NLP-blog-series/blob/main/Day%207%20-%20seq2seq%2C%20encoder-decoder/LanguageTranslation.ipynb